In [1]:
var('t q')

def L_partition_vector2(partition, vector):
    for i in range(len(partition)):
        vector = d[-partition[len(partition) - i - 1]]*vector
    return vector

def my_coefficient(partition, vector):
    return vector.monomial_coefficients().get(tuple(-k for k in partition), 0)

def number_of_ones(partition):
    for i in range(len(partition)):
        if partition[len(partition) - i - 1] != 1:
            return i
    return len(partition)

def my_len(partition):
    return 2*len(partition) - number_of_ones(partition)

def series_t_q(list_of_list_of_partition):
    series = 0
    for i in range(len(list_of_list_of_partition)):
        series_t = 0
        for j in range(len(list_of_list_of_partition[i])):
            series_t = series_t + t^my_len(list_of_list_of_partition[i][j])
        series = series + series_t*q^i
    return series

def partition_ends(partition, tail, is_greater):
    if len(partition) < len(tail) - 1:
        return False
    partition2 = partition[len(partition) - len(tail) + 1: len(partition)]
    if partition2 != tail[1:]:
        return False
    if is_greater:
        if partition == partition2:
            return True
        if partition[len(partition) - len(tail)] > tail[0]:
            return True
    else:
        if partition == partition2:
            return False
        if partition[len(partition) - len(tail)] == tail[0]:
            return True
    return False

def list_of_list_of_partition_ends(list_of_list_of_partition, tail, is_greater):
    return [[partition
             for partition in list_of_partition if partition_ends(partition, tail, is_greater)]
            for list_of_partition in list_of_list_of_partition]

def to_polynomial(partition):
    polynomial = [0]*(sum(partition) - 1)
    for i in range(len(partition) - number_of_ones(partition)):
        polynomial[partition[i] - 2] += 1
    return polynomial

def my_order(partition):
    polynomial = to_polynomial(partition)
    polynomial = [-k for k in polynomial]
    polynomial.reverse()
    return ((my_len(partition), len(partition) - number_of_ones(partition), polynomial)) 

def ordered_partitions(N):
    if N <= 0:
        return [[]]
    list_of_partition = list(Partitions(N))
    list_of_partition.sort(key = my_order, reverse = True)
    return list_of_partition

def is_subpartition(partition, partition2):
    if len(partition) < len(partition2):
        return False
    for i in range(len(partition) - len(partition2) + 1):
        if partition[i: i + len(partition2)] == partition2:
            return True
    return False

def partition_contains_any_of_list(partition, list_of_partition):
    for partition2 in list_of_partition:
        if is_subpartition(partition, partition2):
            return True
    return False

def partition_contains_any_of_R(partition, s, list_of_partition):
    if partition == []:
        return False
    if partition_contains_any_of_list(partition, list_of_partition):
        return True
    if len(partition) < s:
        return False
    for i in range(len(partition) - s + 1):
        if partition[i] - partition[i + s - 1] < 2 and partition[i + s - 1] > 1:
            return True
    return False

def find_singular_vector_at_weight(weight):
    list_of_partition = ordered_partitions(weight)
    list_of_partition.reverse()
    list_of_partition2 = ordered_partitions(weight - 1)
    list_of_partition2.reverse() 
    list_of_partition3 = ordered_partitions(weight - 2)
    list_of_partition3.reverse()
    matrix_1 = matrix({(i, j): my_coefficient(list_of_partition2[i], d[1]*L_partition_vector2(list_of_partition[j], v)) for j in range(len(list_of_partition)) for i in range(len(list_of_partition2))}, sparse = False)
    matrix_2 = matrix({(i, j): my_coefficient(list_of_partition3[i], d[2]*L_partition_vector2(list_of_partition[j], v)) for j in range(len(list_of_partition)) for i in range(len(list_of_partition3))}, sparse = False)
    matrix_1 = matrix_1.stack(matrix_2)
    matrix_1 = matrix_1.rref()
    vector = [-matrix_1[k, len(list_of_partition)-1] for k in range(len(list_of_partition)-1)]
    vector_list = list(reversed(vector + [1]))
    denominators = [value.denominator() for value in vector_list]
    common_denominator = LCM(denominators)
    vector_list = [value*common_denominator for value in vector_list]
    list_of_partition.reverse()
    vector2 = {tuple(list_of_partition[i]): vector_list[i] for i in range(len(list_of_partition))}
    return vector2

def find_singular_vectors():
    return (find_singular_vector_at_weight(m*n), find_singular_vector_at_weight((pp - m)*(qq - n)))

def vector_sum(vector1, vector2):
    return {partition: vector1.get(partition, 0) + vector2.get(partition, 0)
            for partition in set(vector1)|set(vector2)}

def vector_simplify0(partition):
    for i in range(len(partition) - 1):
        if partition[i] < partition[i + 1]:
            return {partition[0: i] + (partition[i] + partition[i + 1],)
                    + partition[i + 2: len(partition)]: partition[i + 1] - partition[i],
                    partition[0: i] + (partition[i + 1],) + (partition[i],)
                    + partition[i + 2: len(partition)]: 1}
    return {partition: 1}

def vector_simplify(vector):
    vector2 = {}
    for partition in set(vector):
        vector3 = vector_simplify0(partition)
        vector2 = vector_sum(vector2, {partition2: vector3[partition2]*vector[partition]
                                       for partition2 in set(vector3)})
    if vector2 == vector:
        return vector2
    return vector_simplify(vector2)

def L_partition_vector(partition, vector):
    return vector_simplify(
        {partition + partition2: vector[partition2] for partition2 in set(vector)})

def matrix_UVir_m1_m2(N):
    singular_vectors = find_singular_vectors()
    m1 = singular_vectors[0]
    m2 = singular_vectors[1]
    list_of_partition = ordered_partitions(N)
    list_of_partition2 = ordered_partitions(N - m*n)
    list_of_partition3 = ordered_partitions(N - (pp - m)*(qq - n))
    list_of_vector_m1 = [L_partition_vector(tuple(list_of_partition2[i]), m1)
                         for i in range(len(list_of_partition2))]
    list_of_vector_m2 = [L_partition_vector(tuple(list_of_partition3[i]), m2)
                         for i in range(len(list_of_partition3))]
    matrix_m1 = matrix({(i, j): list_of_vector_m1[i].get(tuple(list_of_partition[j]), 0)
                        for j in range(len(list_of_partition))
                        for i in range(len(list_of_partition2))}, sparse = False)
    matrix_m2 = matrix({(i, j): list_of_vector_m2[i].get(tuple(list_of_partition[j]), 0)
                        for j in range(len(list_of_partition))
                        for i in range(len(list_of_partition3))}, sparse = False)
    matrix_m1 = matrix_m1.stack(matrix_m2)
    matrix_m1 = matrix_m1.rref()
    return matrix_m1

def pivots_up_to(N):
    list_of_list_of_partition = []
    for i in range(N + 1):
        list_of_partition = ordered_partitions(i)
        list_of_list_of_partition.append(
            [list_of_partition[j] for j in matrix_UVir_m1_m2(i).pivots()])
    return list_of_list_of_partition

def basis_up_to(N):
    list_of_list_of_partition = []
    pivots = pivots_up_to(N)
    for i in range(N + 1):
        list_of_partition = ordered_partitions(i)
        list_of_list_of_partition.append(
            [partition for partition in list_of_partition if partition not in pivots[i]])
    return list_of_list_of_partition

def basis_up_tov2(N, s, list_of_partition):
    list_of_list_of_partition = []
    for i in range(N + 1):
        list_of_partition2 = ordered_partitions(i)
        list_of_list_of_partition.append(
            [partition for partition in list_of_partition2
             if not partition_contains_any_of_R(partition, s, list_of_partition)])
    return list_of_list_of_partition

In [2]:
Vir = lie_algebras.VirasoroAlgebra(QQ)
pp = 2
ss = 3
m = 1
n = 3
R = [[2, 1], [2, 2], [1, 1, 1, 1, 1], [3, 1, 1, 1, 1], [3, 3, 1, 1, 1]]
NN = 25
qq = 2*ss + 1
c = 1 - 6*(pp - qq)^2/(pp*qq)
h = ((n*pp - m*qq)^2 - (pp - qq)^2)/(4*pp*qq)
M = Vir.verma_module(c, h)
d = Vir.basis()
v = M.highest_weight_vector()

print((c, h))
%time w = basis_up_to(NN)

(-68/7, -3/7)
CPU times: user 6min 34s, sys: 3.15 s, total: 6min 37s
Wall time: 6min 37s


In [3]:
%time w2 = basis_up_tov2(NN, ss, R)

CPU times: user 316 ms, sys: 35 µs, total: 316 ms
Wall time: 325 ms


In [4]:
w == w2

True

In [5]:
series_t_q(w)

(2*t^12 + 11*t^11 + 43*t^10 + 41*t^9 + 78*t^8 + 35*t^7 + 48*t^6 + 11*t^5 + 12*t^4 + t^3 + t^2)*q^25 + (t^12 + 7*t^11 + 33*t^10 + 33*t^9 + 68*t^8 + 31*t^7 + 45*t^6 + 10*t^5 + 12*t^4 + t^3 + t^2)*q^24 + (4*t^11 + 23*t^10 + 27*t^9 + 57*t^8 + 28*t^7 + 40*t^6 + 10*t^5 + 11*t^4 + t^3 + t^2)*q^23 + (2*t^11 + 17*t^10 + 21*t^9 + 48*t^8 + 25*t^7 + 37*t^6 + 9*t^5 + 11*t^4 + t^3 + t^2)*q^22 + (t^11 + 10*t^10 + 17*t^9 + 39*t^8 + 22*t^7 + 33*t^6 + 9*t^5 + 10*t^4 + t^3 + t^2)*q^21 + (7*t^10 + 12*t^9 + 33*t^8 + 19*t^7 + 30*t^6 + 8*t^5 + 10*t^4 + t^3 + t^2)*q^20 + (3*t^10 + 9*t^9 + 25*t^8 + 17*t^7 + 26*t^6 + 8*t^5 + 9*t^4 + t^3 + t^2)*q^19 + (2*t^10 + 6*t^9 + 20*t^8 + 14*t^7 + 24*t^6 + 7*t^5 + 9*t^4 + t^3 + t^2)*q^18 + (4*t^9 + 15*t^8 + 12*t^7 + 20*t^6 + 7*t^5 + 8*t^4 + t^3 + t^2)*q^17 + (2*t^9 + 11*t^8 + 10*t^7 + 18*t^6 + 6*t^5 + 8*t^4 + t^3 + t^2)*q^16 + (t^9 + 7*t^8 + 8*t^7 + 15*t^6 + 6*t^5 + 7*t^4 + t^3 + t^2)*q^15 + (5*t^8 + 6*t^7 + 13*t^6 + 5*t^5 + 7*t^4 + t^3 + t^2)*q^14 + (2*t^8 + 5*t^7 + 10*t^